In [1]:
# -*- coding: utf-8 -*-
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import tensorflow as tf
import struct 
import numpy as np

from PIL import Image
import matplotlib.pyplot as plt

import argparse



from datetime import datetime
import time

import cifar10

#### 这是客户端本地项目路径  映射到   /code

In [3]:
!ls

BUILD		     cifar10_input_test.py	 __init__.py
cifar-10-batches-py  cifar10_log		 predict.es.ipynb
cifar10_data	     cifar10_multi_gpu_train.py  __pycache__
cifar10_eval	     cifar10.py			 README.md
cifar10_eval.py      cifar10_train		 train.cifar.ipynb
cifar10_input.py     cifar10_train.py


In [4]:
!ls cifar10_data/cifar-10-batches-bin

batches.meta.txt  data_batch_2.bin  data_batch_4.bin  readme.html
data_batch_1.bin  data_batch_3.bin  data_batch_5.bin  test_batch.bin


#### 先下载数据库
     客户端执行  polyaxon run -f job.yml  然后会在  虚拟主机/data/mnist在有数据文件

####  数据路径存在虚拟主机路径下： /data/mnist 注意，客户端需要执行 polyaxon run -f job.yml

In [5]:
!ls /data/cifar10
#data_path = '/data/cifar10/'

ls: cannot access '/data/cifar10': No such file or directory


In [6]:
train_path_local = './cifar10_log/'

In [12]:
#模型参数
max_steps =100
#train_dir   = data_path_local
log_device_placement = False

log_frequency = 10

In [7]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('train_dir', train_path_local,
                           """Directory where to write event logs """
                           """and checkpoint.""")
tf.app.flags.DEFINE_integer('max_steps', 100000,
                            """Number of batches to run.""")
tf.app.flags.DEFINE_boolean('log_device_placement', False,
                            """Whether to log device placement.""")
tf.app.flags.DEFINE_integer('log_frequency', 10,
                            """How often to log results to the console.""")

In [8]:
def set_logging(log_level=None):
    if log_level == 'INFO':
        log_level = tf.logging.INFO
    elif log_level == 'DEBUG':
        log_level = tf.logging.DEBUG
    elif log_level == 'WARN':
        log_level = tf.logging.WARN
    else:
        log_level = 'INFO'

    tf.logging.set_verbosity(log_level)

In [9]:
set_logging('DEBUG')


In [10]:
#定义模型网络  tf.esstimator api
def train():
  """Train CIFAR-10 for a number of steps."""
  with tf.Graph().as_default():

    global_step = tf.train.get_or_create_global_step()

    # Get images and labels for CIFAR-10.
    # Force input pipeline to CPU:0 to avoid operations sometimes ending up on
    # GPU and resulting in a slow down.
    with tf.device('/cpu:0'):
      images, labels = cifar10.distorted_inputs()

    # Build a Graph that computes the logits predictions from the
    # inference model.
    logits = cifar10.inference(images)

    # Calculate loss.
    loss = cifar10.loss(logits, labels)

    # Build a Graph that trains the model with one batch of examples and
    # updates the model parameters.
    train_op = cifar10.train(loss, global_step)

    class _LoggerHook(tf.train.SessionRunHook):
      """Logs loss and runtime."""

      def begin(self):
        self._step = -1
        self._start_time = time.time()

      def before_run(self, run_context):
        self._step += 1
        return tf.train.SessionRunArgs(loss)  # Asks for loss value.

      def after_run(self, run_context, run_values):
        if self._step % FLAGS.log_frequency == 0:
          current_time = time.time()
          duration = current_time - self._start_time
          self._start_time = current_time

          loss_value = run_values.results
          examples_per_sec = FLAGS.log_frequency * FLAGS.batch_size / duration
          sec_per_batch = float(duration / FLAGS.log_frequency)

          format_str = ('%s: step %d, loss = %.2f (%.1f examples/sec; %.3f '
                        'sec/batch)')
          print (format_str % (datetime.now(), self._step, loss_value,
                               examples_per_sec, sec_per_batch))

    with tf.train.MonitoredTrainingSession(
        checkpoint_dir=FLAGS.train_dir,
        hooks=[tf.train.StopAtStepHook(last_step=FLAGS.max_steps),
               tf.train.NanTensorHook(loss),
               _LoggerHook()],
        config=tf.ConfigProto(
            log_device_placement=FLAGS.log_device_placement)) as mon_sess:
      while not mon_sess.should_stop():
        mon_sess.run(train_op)



In [13]:
#解决jupyter下cifar10 报错 UnrecognizedFlagError: Unknown command line flag 'f'
tf.app.flags.DEFINE_string('f', '', 'kernel')

In [14]:
cifar10.maybe_download_and_extract()

In [ ]:
cifar10.maybe_download_and_extract()

if tf.gfile.Exists(FLAGS.train_dir):
tf.gfile.DeleteRecursively(FLAGS.train_dir)

tf.gfile.MakeDirs(FLAGS.train_dir)

train()

In [15]:
train()

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Filling queue with 20000 CIF

2019-03-31 12:46:42.267058: step 480, loss = 3.53 (7023.6 examples/sec; 0.018 sec/batch)
2019-03-31 12:46:42.427942: step 490, loss = 3.18 (7956.3 examples/sec; 0.016 sec/batch)
INFO:tensorflow:global_step/sec: 50.4138
2019-03-31 12:46:42.855181: step 500, loss = 3.10 (2995.9 examples/sec; 0.043 sec/batch)
2019-03-31 12:46:43.031383: step 510, loss = 3.33 (7265.0 examples/sec; 0.018 sec/batch)
2019-03-31 12:46:43.200507: step 520, loss = 3.18 (7568.2 examples/sec; 0.017 sec/batch)
2019-03-31 12:46:43.378384: step 530, loss = 2.95 (7195.9 examples/sec; 0.018 sec/batch)
2019-03-31 12:46:43.586950: step 540, loss = 3.03 (6137.0 examples/sec; 0.021 sec/batch)
2019-03-31 12:46:43.807842: step 550, loss = 3.18 (5795.2 examples/sec; 0.022 sec/batch)
2019-03-31 12:46:43.978373: step 560, loss = 3.01 (7505.2 examples/sec; 0.017 sec/batch)
2019-03-31 12:46:44.146219: step 570, loss = 3.10 (7626.3 examples/sec; 0.017 sec/batch)
2019-03-31 12:46:44.321524: step 580, loss = 3.01 (7301.5 examples/se

2019-03-31 12:47:00.040321: step 1360, loss = 2.25 (7356.8 examples/sec; 0.017 sec/batch)
2019-03-31 12:47:00.202001: step 1370, loss = 2.02 (7917.2 examples/sec; 0.016 sec/batch)
2019-03-31 12:47:00.370013: step 1380, loss = 2.17 (7618.9 examples/sec; 0.017 sec/batch)
2019-03-31 12:47:00.535938: step 1390, loss = 2.11 (7714.0 examples/sec; 0.017 sec/batch)
INFO:tensorflow:global_step/sec: 51.991
2019-03-31 12:47:00.933899: step 1400, loss = 2.16 (3216.5 examples/sec; 0.040 sec/batch)
2019-03-31 12:47:01.107594: step 1410, loss = 2.00 (7368.5 examples/sec; 0.017 sec/batch)
2019-03-31 12:47:01.273635: step 1420, loss = 1.85 (7709.3 examples/sec; 0.017 sec/batch)
2019-03-31 12:47:01.438721: step 1430, loss = 2.05 (7753.5 examples/sec; 0.017 sec/batch)
2019-03-31 12:47:01.605309: step 1440, loss = 2.00 (7683.9 examples/sec; 0.017 sec/batch)
2019-03-31 12:47:01.772210: step 1450, loss = 1.94 (7669.0 examples/sec; 0.017 sec/batch)
2019-03-31 12:47:01.940320: step 1460, loss = 1.98 (7614.2 e

2019-03-31 12:47:16.898577: step 2240, loss = 1.68 (7311.1 examples/sec; 0.018 sec/batch)
2019-03-31 12:47:17.065056: step 2250, loss = 1.62 (7688.7 examples/sec; 0.017 sec/batch)
2019-03-31 12:47:17.232799: step 2260, loss = 1.50 (7630.6 examples/sec; 0.017 sec/batch)
2019-03-31 12:47:17.400664: step 2270, loss = 1.61 (7625.4 examples/sec; 0.017 sec/batch)
2019-03-31 12:47:17.571373: step 2280, loss = 1.37 (7497.9 examples/sec; 0.017 sec/batch)
2019-03-31 12:47:17.734745: step 2290, loss = 1.32 (7835.2 examples/sec; 0.016 sec/batch)
INFO:tensorflow:global_step/sec: 52.6161
2019-03-31 12:47:18.118245: step 2300, loss = 1.67 (3338.0 examples/sec; 0.038 sec/batch)
2019-03-31 12:47:18.291559: step 2310, loss = 1.40 (7383.8 examples/sec; 0.017 sec/batch)
2019-03-31 12:47:18.461819: step 2320, loss = 1.55 (7518.2 examples/sec; 0.017 sec/batch)
2019-03-31 12:47:18.628056: step 2330, loss = 1.48 (7699.9 examples/sec; 0.017 sec/batch)
2019-03-31 12:47:18.797324: step 2340, loss = 1.55 (7561.7 

2019-03-31 12:47:34.855766: step 3120, loss = 1.34 (7076.5 examples/sec; 0.018 sec/batch)
2019-03-31 12:47:35.065668: step 3130, loss = 1.31 (6098.4 examples/sec; 0.021 sec/batch)
2019-03-31 12:47:35.299659: step 3140, loss = 1.29 (5470.3 examples/sec; 0.023 sec/batch)
2019-03-31 12:47:35.533740: step 3150, loss = 1.15 (5468.1 examples/sec; 0.023 sec/batch)
2019-03-31 12:47:35.755483: step 3160, loss = 1.06 (5773.4 examples/sec; 0.022 sec/batch)
2019-03-31 12:47:35.968117: step 3170, loss = 1.20 (6018.5 examples/sec; 0.021 sec/batch)
2019-03-31 12:47:36.181463: step 3180, loss = 1.27 (5999.6 examples/sec; 0.021 sec/batch)
2019-03-31 12:47:36.358935: step 3190, loss = 1.16 (7212.7 examples/sec; 0.018 sec/batch)
INFO:tensorflow:global_step/sec: 43.3796
2019-03-31 12:47:36.806107: step 3200, loss = 1.38 (2862.7 examples/sec; 0.045 sec/batch)
2019-03-31 12:47:36.982865: step 3210, loss = 1.29 (7240.1 examples/sec; 0.018 sec/batch)
2019-03-31 12:47:37.154906: step 3220, loss = 1.26 (7439.8 

KeyboardInterrupt: 

In [16]:
!ls {train_path_local}

checkpoint			    model.ckpt-0.data-00001-of-00002
events.out.tfevents.1554007577.zkl  model.ckpt-0.index
graph.pbtxt			    model.ckpt-0.meta
model.ckpt-0.data-00000-of-00002
